In [1]:
import os
PINECONE_API_KEY = "proide-your-own-key"
GROQ_API_KEY = "proide-your-own-key"
COHERE_API_KEY = "proide-your-own-key"
HF_TOKEN= "proide-your-own-key"

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY']=GROQ_API_KEY
os.environ['COHERE_API_KEY'] = COHERE_API_KEY


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install langchain_huggingface pinecone-client[grpc] cohere langchain langchain-community langchain-groq langchain-pinecone faiss-cpu pypdf lxml beautifulsoup4 sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key = PINECONE_API_KEY)

In [5]:
from pinecone import PodSpec

environment = 'gcp-starter'
index_name = 'ki-doctor'

spec = PodSpec(environment=environment)

In [6]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.05747,
 'namespaces': {'': {'vector_count': 5747}},
 'total_vector_count': 5747}

In [ ]:
pc.delete_index(index_name)

In [7]:
import xml.etree.ElementTree as ET
import re

def extract_text_from_recommendation_sections(input_file):
    # Parse the XML file
    tree = ET.parse(input_file)
    root = tree.getroot()

    # Find all <recommendation> elements
    recommendation_elements = root.findall('.//recommendation')

    # List to hold the extracted text content
    text_list = []

    i = 0

    # Iterate through all <recommendation> elements
    for recommendation_element in recommendation_elements:
        # Get the content of the <recommendation> element as a string
        recommendation = recommendation_element.findall('.//text')

        # Add the text to the list
        text_list.append(recommendation[0].text)

    return text_list

# Example usage
input_file = '/content/drive/MyDrive/cpg-corpus-cms.xml'
text_list = extract_text_from_recommendation_sections(input_file)

# Print out the first bottom section's text for demonstration
print(len(text_list))
print(text_list[11])


5747

      Bei zu erwartenden Metallartefakten im Bereich der Mundhöhle sollte die MRT dem CT zur Beurteilung des Primärtumors vorgezogen werden.
     


In [10]:
from langchain.docstore.document import Document

# Create document objects
docs = [Document(page_content=text, metadata={"source": "local"}) for text in text_list] #TODO: Change "local" to actual source of respective text
print(f"Number of documents: {len(docs)}")

Number of documents: 5747


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    )


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    docs,
    embeddings,
    index_name=index_name
)


In [12]:
import math

# Define the total limit for chunks
TOTAL_CHUNKS_LIMIT = 9999


# Calculate the maximum number of chunks per document
max_chunks_per_doc = TOTAL_CHUNKS_LIMIT // len(docs)

# Function to chunk a document into smaller parts
def chunk_document(doc, max_chunks):
    chunk_size = max(1, math.ceil(len(doc) / max_chunks))
    return [doc[i:i+chunk_size] for i in range(0, len(doc), chunk_size)]

# Process the documents into chunks
chunked = []
for doc in docs:
    chunked.extend(chunk_document(doc.page_content, max_chunks_per_doc))

# Check the total number of chunks
total_chunks = len(chunked)
print(f"Total number of chunks: {total_chunks}")

# Ensure the total number of chunks does not exceed the limit
assert total_chunks <= TOTAL_CHUNKS_LIMIT, "Total number of chunks exceeds the limit"

# Now chunked_docs can be used with Cohere's rerank Client


Total number of chunks: 5747


In [13]:
import cohere
co = cohere.Client(api_key=COHERE_API_KEY)

query = "Was sind Risikofaktoren für ein Mundhöhlenkarzinom?"

co_doc = co.rerank(
    model='rerank-multilingual-v3.0',
    query = query,
    documents = chunked[1:1000],
    top_n = 20,
    return_documents= True
)

In [14]:
co_doc.results[1]

RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='\n      Wichtige Risikofaktoren für das Nicht-Kardiakarzinom sind Alter, niedriger sozioökonomischer Status, Tabakrauchen, Alkoholkonsum, familiäre Belastung, vorangegangene Magenoperationen, perniziöse Anämie, Leben in einer Hochrisikopopulation sowie Ernährungs- und Umweltfaktoren.\n     '), index=97, relevance_score=0.8597815)

In [15]:

res = vectorstore.similarity_search(
    query,
    k=250
)

result_doc = []
for result in res:
    result_doc.append(result.page_content)

print(result_doc[0:5])


['\n      Tabakkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.\n     ', '\n      Tabakkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.\n     ', '\n      Alkoholkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.\n     ', '\n      Alkoholkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.\n     ', '\n      Bei Vorhandensein von Risikofaktoren sollte das Vorliegen eines KZT abgeklärt werden.\n     ']


In [17]:
co_doc = co.rerank(
    model='rerank-multilingual-v3.0',
    query = query,
    documents = result_doc,
    top_n = 20,
    return_documents= True
)

co_doc.results[0]

RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='\n     Bei Verdacht auf das Vorliegen eines Analkarzinoms soll eine ausführliche Anamnese auch zur Abklärung von Risikofaktoren (Immundefizienz einschließlich HIV-Infektion, rezeptiver Analverkehr, Vorerkrankung mit HPV-assoziierten anogenitalen Läsionen, Nikotinabusus) erhoben werden.\n    '), index=144, relevance_score=0.24816959)

In [18]:
matched_info = '\n'.join(item.document.text for item in co_doc.results)
print(matched_info)
sources = [item.metadata for item in res]
context = f"{matched_info}"


sys_prompt = f"""
Du bist ein Arzt und wirst von einem Patienten etwas gefragt. Du bekommst außerdem einen Kontext von relevanten medizinisch genauen Aussagen, welche du verwenden sollst
um eine Antwort zu formulieren.
Anweisungen:
- Sei hilfreich und beantworte die Frage so genau wie möglich.
- Nutze den Kontext der dir gegeben wird für genaue und spezielle Informationen
- Antworte auf einfache Fragen so kurz und einfach wie möglich, nutze hierfür nur die relevantesten Teile des dir gegebenen Kontexts
- Gib für umfangreichere Fragen mehr Informationen aus dem dir gegebenen Kontext
Kontext: {context}
Frage:{query}
"""


      Alkoholkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.
     

      Alkoholkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.
     

      Tabakkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.
     

      Tabakkonsum ist ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms.
     

      Wichtige Risikofaktoren für das Nicht-Kardiakarzinom sind Alter, niedriger sozioökonomischer Status, Tabakrauchen, Alkoholkonsum, familiäre Belastung, vorangegangene Magenoperationen, perniziöse Anämie, Leben in einer Hochrisikopopulation sowie Ernährungs- und Umweltfaktoren.
     

      Wichtige Risikofaktoren für das Nicht-Kardiakarzinom sind Alter, niedriger sozioökonomischer Status, Tabakrauchen, Alkoholkonsum, familiäre Belastung, vorangegangene Magenoperationen, perniziöse Anämie, Leben in einer Hochrisikopopulation sowie Ernährungs- und Umweltfaktoren.
     


In [19]:
from groq import Groq

client = Groq(
    api_key="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"
)

chat_completion = client.chat.completions.create(
    messages = [
        {
            "role":"user",
            "content":f"{sys_prompt}"
        }
    ],
    model = "mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

Lieber Patient,

danke, dass Sie mich konsultiert haben. Ein wesentlicher Risikofaktor für die Entwicklung des Mundhöhlenkarzinoms sind Alkoholkonsum und Tabakkonsum. Der Alkoholkonsum und der Tabakkonsum können entweder einzeln oder in Kombination auftreten und das Risiko für Mundhöhlenkarzinom signifikant erhöhen. Eine verbesserte Aufklärung über Anzeichen, Symptome und Risikofaktoren des Mundhöhlenkarzinoms kann bei der Früherkennung und Behandlung dieser Erkrankung hilfreich sein.

Ich hoffe, ich konnte Ihre Frage zufriedenstellend beantworten. Wenn Sie noch weitere Fragen haben, zögern Sie bitte nicht, mich zu kontaktieren.

Mit freundlichen Grüßen,
[Ihr Name]


In [ ]:
def qa(question):
    res = vectorstore.similarity_search(
        question,
        k=250
    )

    result_doc = []
    for result in res:
        result_doc.append(result.page_content)

    co_doc = co.rerank(
    model='rerank-multilingual-v3.0',
    query = question,
    documents = result_doc,
    top_n = 25,
    return_documents= True
    )

    matched_info = '\n'.join(item.document.text for item in co_doc.results)
    context = f"{matched_info}"
    sys_prompt = f"""
        Anweisungen:
        - Sei hilfreich und beantworte die Frage so genau wie möglich.
        - Nutze den Kontext der dir gegeben wird für genaue und spezielle Informationen
        - Antworte auf einfache Fragen so kurz und einfach wie möglich, nutze hierfür nur die relevantesten Teile des dir gegebenen Kontexts
        - Gib für umfangreichere Fragen mehr Informationen aus dem dir gegebenen Kontext
        Kontext: {context}
        Frage:{question}
        """

    chat_completion = client.chat.completions.create(
        messages = [
            {
            "role":"user",
            "content":f"{sys_prompt}"
            }
        ],
        model = "mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content




In [ ]:
question = "Was sind Risikofaktoren für ein Mundhöhlenkarzinom?"
question2 = "Was sind neben Alkoholkonsum und Tabakkonsum weitere Risikofaktoren für ein Mundhöhlenkarzinom?"
print(qa(question))
print("")
print(qa(question2))

Risikofaktoren für ein Mundhöhlenkarzinom sind Alkoholkonsum und Tabakkonsum.

Neben Alkoholkonsum und Tabakkonsum sind weitere Risikofaktoren für ein Mundhöhlenkarzinom ein niedriger sozioökonomischer Status, eine familiäre Belastung, vorangegangene Magenoperationen, perniziöse Anämie, Leben in einer Hochrisikopopulation, Ernährungs- und Umweltfaktoren.
